In [1]:
from fastai2.text.all import *
from pigboat.text.all import *

In [2]:
source = untar_data(URLs.IMDB_SAMPLE)

In [3]:
df = pd.read_csv(source/'texts.csv')
df.head(1)

,label,text,is_valid
0,negative,"Un-bleeping-believable! Meg Ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. Hard to believe she was the producer on this dog. Plus Kevin Kline: what kind of suicide trip has his career been on? Whoosh... Banzai!!! Finally this was directed by the guy who did Big Chill? Must be a replay of Jonestown - hollywood style. Wooofff!",False


In [4]:
df['text2'] = df['text']

In [5]:
ABSTAIN,POS,NEG = 'abstain','positive','negative'

In [6]:
labeller = Labeller(ABSTAIN)

In [7]:
@labeller(ColReader)
def liked(x): return POS if 'liked' in x else None
@labeller(ColReader)
def hated(x): return NEG if 'hated' in x else None

In [8]:
splits = ColSplitter()(df)

In [9]:
#TODO: How to not pass ABSTAIN?
tls = TfmdLists(df, [ColReader('text'), Tokenizer.from_df('text2'), Numericalize,
                     CategorizeTaskLabels([ABSTAIN,POS,NEG])])

In [10]:
tls[0]

TensorText([   2,    8, 4499,   24,    0,   24,  951,   55,    8, 4500,    8, 1769,
          95,   37,   78,  198,   62,  852,    0, 2184,  507,   18,   21,   10,
          86, 2035,  192,   90, 3814,   62, 1895,    0,  124,    0,   11,    8,
         251,   15,  277,   73,   25,    9, 1651,   35,   21, 1138,   11,    8,
        1096,    8, 1353,    8, 2921,   92,   64,  240,   14, 1576, 1484,   66,
          41,  790,  107,   35,   69,    8,    0,   87,    8,    0,    5,  161,
          55,    8,  567,   21,   25,  642,   48,    9,  265,   53,   89,    8,
         218,    8,    0,   69,    8,  234,   43,   13, 4501,   14,    8, 4502,
          24,  392,  427,   11, 2627,    5,  161, 1354,    5,  161, 1288,   55])